# PC 3 

### Pregunta 2

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

Importamos la data de carseats.csv

In [3]:
data = pd.read_csv('C:/Users/arica/OneDrive/Documentos/GitHub/Qlab_Repositorio/Introducción a Machine Learning/PC3/carseats.csv')

data

,Unnamed: 0,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


In [4]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X = data.drop('Sales', axis=1)  # independientes
y = data['Sales']  # dependiente

# 70% entrenamiento, 30% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Observamos primeras filas
data.head(), X_train.shape, X_test.shape

(   Unnamed: 0  Sales  CompPrice  Income  Advertising  Population  Price  \
 0           0   9.50        138      73           11         276    120   
 1           1  11.22        111      48           16         260     83   
 2           2  10.06        113      35           10         269     80   
 3           3   7.40        117     100            4         466     97   
 4           4   4.15        141      64            3         340    128   
 
   ShelveLoc  Age  Education Urban   US  
 0       Bad   42         17   Yes  Yes  
 1      Good   65         10   Yes  Yes  
 2    Medium   59         12   Yes  Yes  
 3    Medium   55         14   Yes  Yes  
 4       Bad   38         13   Yes   No  ,
 (280, 11),
 (120, 11))

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identificamos las columnas categóricas
categorical_features = data.select_dtypes(include=['object']).columns

# Creamos un transformador de columnas para la codificación de las variables categóricas
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Creamos un árbol de regresión con el transformador de columnas
regressor = Pipeline(steps=[('preprocessor', column_transformer),
                            ('regressor', DecisionTreeRegressor(random_state=42))])

# Ajustamos el modelo al conjunto de entrenamiento
regressor.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred = regressor.predict(X_test)

# Calculamos el MSE para el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
mse



4.7374616666666665

El modelo de árbol de regresión ha sido ajustado al conjunto de entrenamiento. El valor del Mean Squared Error (MSE) para el conjunto de prueba es aproximadamente 4.737.

Ahora, pasemos al tercer paso: aplicar el método Bagging y calcular el MSE para el conjunto de prueba.

El modelo Bagging ha sido ajustado y el valor del Mean Squared Error (MSE) para el conjunto de prueba es aproximadamente 2.687, lo cual es una mejora en comparación con el árbol de regresión solo.

In [7]:
# Creamos un modelo Bagging con árboles de decisión como base
bagging_regressor = Pipeline(steps=[('preprocessor', column_transformer),
                                    ('regressor', BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=42),
                                                                   random_state=42))])

# Ajustamos el modelo Bagging al conjunto de entrenamiento
bagging_regressor.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred_bagging = bagging_regressor.predict(X_test)

# Calculamos el MSE para el conjunto de prueba con Bagging
mse_bagging = mean_squared_error(y_test, y_pred_bagging)
mse_bagging



c:\Users\arica\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


2.686782216666666

Random Forest:

El modelo de Random Forest ha sido ajustado y el valor del Mean Squared Error (MSE) para el conjunto de prueba es aproximadamente 2.612, lo cual indica una mejora adicional en comparación con el modelo Bagging.

Las cinco características más importantes en el modelo de Random Forest son:

Price: con la mayor importancia.
ShelveLoc_Good: ubicación del producto en la tienda.
Age: edad del consumidor.
CompPrice: precio de la competencia.
Advertising: gastos en publicidad.

In [8]:
# Creando un modelo Random Forest
random_forest_regressor = Pipeline(steps=[('preprocessor', column_transformer),
                                          ('regressor', RandomForestRegressor(random_state=42))])

# Ajustando el modelo Random Forest al conjunto de entrenamiento
random_forest_regressor.fit(X_train, y_train)

# Realizando predicciones en el conjunto de prueba
y_pred_rf = random_forest_regressor.predict(X_test)

# Calculando el MSE para el conjunto de prueba con Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)

# Obteniendo la importancia de las características
feature_importances = random_forest_regressor.named_steps['regressor'].feature_importances_

# Creando un DataFrame para visualizar mejor las importancias de las características
features = column_transformer.transformers_[0][1].get_feature_names_out().tolist() + X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Ordenando las características por importancia
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)

mse_rf, feature_importance_df.head()  # Mostrando el MSE y las principales características



(2.6122368601666657,
            Feature  Importance
 12           Price    0.304137
 1   ShelveLoc_Good    0.177810
 13             Age    0.117862
 8        CompPrice    0.093678
 10     Advertising    0.067988)

Gradient Boosting

El modelo de Gradient Boosting ha sido ajustado y el valor del MSE para el conjunto de prueba es aproximadamente 2.159, lo cual muestra una mejora notable en comparación con los modelos anteriores.

Las cinco características más importantes en el modelo de Gradient son:

Price: con la mayor importancia.
ShelveLoc_Good: ubicación del producto en la tienda.
CompPrice: precio de la competencia.
Age: edad del consumidor.
ShelveLoc_Bad: otra categoría de la ubicación del producto en la tienda.

In [10]:
# Creamos un modelo Gradient Boosting
gradient_boosting_regressor = Pipeline(steps=[('preprocessor', column_transformer),
                                              ('regressor', GradientBoostingRegressor(random_state=42))])

# Ajustamos el modelo Gradient Boosting al conjunto de entrenamiento
gradient_boosting_regressor.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba
y_pred_gb = gradient_boosting_regressor.predict(X_test)

# Calculamos el MSE para el conjunto de prueba con Gradient Boosting
mse_gb = mean_squared_error(y_test, y_pred_gb)

# Obtenemos la importancia de las características en Gradient Boosting
feature_importances_gb = gradient_boosting_regressor.named_steps['regressor'].feature_importances_

# Creamos un DataFrame para las características en Gradient Boosting
feature_importance_gb_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances_gb})

# Ordenamos las características por importancia
feature_importance_gb_df.sort_values(by='Importance', ascending=False, inplace=True)

mse_gb, feature_importance_gb_df.head()  # Mostrando el MSE y las principales características en Gradient Boosting




(2.158555043476713,
            Feature  Importance
 12           Price    0.338252
 1   ShelveLoc_Good    0.174057
 8        CompPrice    0.119547
 13             Age    0.112794
 0    ShelveLoc_Bad    0.079337)